In [13]:
import requests
import json
import urllib 
import datetime as dt
from pandas import read_excel
from pandas import read_csv
from pandas import concat
from pandas import merge
import pandas as pd
import numpy as np
from haversine import haversine
from pandas import Series
from pandas import DataFrame

api_key = '0939d46ec028e873d9948042fe319c5b'

user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"
header_info = {'User-agent': user_agent, 'referer': None, 'Authorization': 'KakaoAK ' + api_key}

In [16]:
def get_pos(q):

    params = {'query' : q, 'analyze_type' : 'exact'}
    query = urllib.parse.urlencode(params)

    url_tpl = 'https://dapi.kakao.com/v2/local/search/address.json'
    api_url = url_tpl + '?' + query
    r = requests.get(api_url, headers=header_info)

    r.encoding = 'utf-8'
    address_dict = json.loads(r.text)
    address_dict
    
    try:
        좌표 = (address_dict['documents'][0]['address']['x'], address_dict['documents'][0]['address']['y'])
        return 좌표
    except:
        return q

In [17]:
apt_value = read_excel('아파트(매매)_실거래가_20210802122921.xlsx', engine='openpyxl')
apt_value

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일
0,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,202104,15,174000,9,1988,언주로 103,NaN
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,202105,15,195000,5,1987,언주로 3,NaN
2,서울특별시 강남구 개포동,1282,1282.0,0.0,개포래미안포레스트,49.92,202105,22,180000,7,2020,개포로 264,20210526.0
3,서울특별시 강남구 개포동,1282,1282.0,0.0,개포래미안포레스트,59.92,202105,22,198500,18,2020,개포로 264,20210526.0
4,서울특별시 강남구 개포동,1282,1282.0,0.0,개포래미안포레스트,59.92,202106,13,210000,5,2020,개포로 264,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28593,서울특별시 중랑구 중화동,450,450.0,0.0,한신2,59.76,202104,10,66000,6,1998,동일로 752,NaN
28594,서울특별시 중랑구 중화동,450,450.0,0.0,한신2,59.76,202104,16,68000,13,1998,동일로 752,NaN
28595,서울특별시 중랑구 중화동,274-51,274.0,51.0,한영(101),57.66,202102,27,30500,5,2003,동일로144길 74,NaN
28596,서울특별시 중랑구 중화동,274-75,274.0,75.0,한영(102),72.22,202105,4,49500,4,2003,동일로144길 74,NaN


In [18]:
road_name = apt_value['도로명']

posX = []
posY = []
non_get_pos = []

for i,v in enumerate(road_name):
    pos = get_pos(v)
    
    if type(pos) == str:
        non_get_pos.append((i,v))
        posX.append(None)
        posY.append(None)
    else:
        posX.append(pos[0])
        posY.append(pos[1])

In [19]:
apt_value['경도'] = posX
apt_value['위도'] = posY
apt_value

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,경도,위도
0,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,202104,15,174000,9,1988,언주로 103,NaN,127.052911761665,37.481343842076
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,202105,15,195000,5,1987,언주로 3,NaN,127.055925867398,37.4767921771121
2,서울특별시 강남구 개포동,1282,1282.0,0.0,개포래미안포레스트,49.92,202105,22,180000,7,2020,개포로 264,20210526.0,127.052228213432,37.4797503513842
3,서울특별시 강남구 개포동,1282,1282.0,0.0,개포래미안포레스트,59.92,202105,22,198500,18,2020,개포로 264,20210526.0,127.052228213432,37.4797503513842
4,서울특별시 강남구 개포동,1282,1282.0,0.0,개포래미안포레스트,59.92,202106,13,210000,5,2020,개포로 264,NaN,127.052228213432,37.4797503513842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28593,서울특별시 중랑구 중화동,450,450.0,0.0,한신2,59.76,202104,10,66000,6,1998,동일로 752,NaN,127.082070702444,37.5970090851956
28594,서울특별시 중랑구 중화동,450,450.0,0.0,한신2,59.76,202104,16,68000,13,1998,동일로 752,NaN,127.082070702444,37.5970090851956
28595,서울특별시 중랑구 중화동,274-51,274.0,51.0,한영(101),57.66,202102,27,30500,5,2003,동일로144길 74,NaN,127.08216473131,37.6065211349668
28596,서울특별시 중랑구 중화동,274-75,274.0,75.0,한영(102),72.22,202105,4,49500,4,2003,동일로144길 74,NaN,127.08216473131,37.6065211349668


In [20]:
apt_value['경도'] = apt_value['경도'].astype('float')
apt_value['위도'] = apt_value['위도'].astype('float')
apt_value.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28598 entries, 0 to 28597
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시군구       28598 non-null  object 
 1   번지        28583 non-null  object 
 2   본번        28594 non-null  float64
 3   부번        28594 non-null  float64
 4   단지명       28598 non-null  object 
 5   전용면적(㎡)   28598 non-null  float64
 6   계약년월      28598 non-null  int64  
 7   계약일       28598 non-null  int64  
 8   거래금액(만원)  28598 non-null  int64  
 9   층         28598 non-null  int64  
 10  건축년도      28598 non-null  int64  
 11  도로명       28598 non-null  object 
 12  해제사유발생일   681 non-null    float64
 13  경도        28523 non-null  float64
 14  위도        28523 non-null  float64
dtypes: float64(6), int64(5), object(4)
memory usage: 3.3+ MB


In [21]:
station_pos = read_csv('station_coordinate.csv')
station_pos

,line,name,code,lat,lng
0,01호선,녹양,1908.0,37.759380,127.042292
1,01호선,남영,1002.0,37.541021,126.971300
2,01호선,용산,1003.0,37.529849,126.964561
3,01호선,노량진,1004.0,37.514219,126.942454
4,01호선,대방,1005.0,37.513342,126.926382
...,...,...,...,...,...
725,인천선,지식정보단지,3135.0,37.378384,126.645168
726,인천선,인천대입구,3136.0,37.386007,126.639484
727,인천선,센트럴파크,3137.0,37.393054,126.634729
728,인천선,국제업무지구,3138.0,37.399907,126.630347


In [22]:
near_station = []
for i in apt_value.index:
    start = (apt_value.loc[i, '위도'], apt_value.loc[i, '경도'])
    distance = []
    for j in station_pos.index:
        try:
            end = (station_pos.loc[j, 'lat'],station_pos.loc[j,'lng'])
            distance.append([haversine(start,end),station_pos.loc[j,'line'],station_pos.loc[j,'name']])
        except:
            distance.append([np.nan,np.nan,np.nan])
    near_station.append(min(distance))
        

In [23]:
near_df = DataFrame(near_station,columns=['역과의거리(km)', '호선', '역'])
near_df

,역과의거리(km),호선,역
0,0.805269,분당선,구룡
1,1.146686,분당선,구룡
2,0.934028,03호선,매봉
3,0.934028,03호선,매봉
4,0.934028,03호선,매봉
...,...,...,...
28593,0.270638,07호선,상봉
28594,0.270638,07호선,상봉
28595,0.510661,07호선,중화
28596,0.510661,07호선,중화


In [24]:
mdf = merge(apt_value, near_df,left_index= True, right_index=True)
mdf

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,경도,위도,역과의거리(km),호선,역
0,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,202104,15,174000,9,1988,언주로 103,NaN,127.052912,37.481344,0.805269,분당선,구룡
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,202105,15,195000,5,1987,언주로 3,NaN,127.055926,37.476792,1.146686,분당선,구룡
2,서울특별시 강남구 개포동,1282,1282.0,0.0,개포래미안포레스트,49.92,202105,22,180000,7,2020,개포로 264,20210526.0,127.052228,37.479750,0.934028,03호선,매봉
3,서울특별시 강남구 개포동,1282,1282.0,0.0,개포래미안포레스트,59.92,202105,22,198500,18,2020,개포로 264,20210526.0,127.052228,37.479750,0.934028,03호선,매봉
4,서울특별시 강남구 개포동,1282,1282.0,0.0,개포래미안포레스트,59.92,202106,13,210000,5,2020,개포로 264,NaN,127.052228,37.479750,0.934028,03호선,매봉
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28593,서울특별시 중랑구 중화동,450,450.0,0.0,한신2,59.76,202104,10,66000,6,1998,동일로 752,NaN,127.082071,37.597009,0.270638,07호선,상봉
28594,서울특별시 중랑구 중화동,450,450.0,0.0,한신2,59.76,202104,16,68000,13,1998,동일로 752,NaN,127.082071,37.597009,0.270638,07호선,상봉
28595,서울특별시 중랑구 중화동,274-51,274.0,51.0,한영(101),57.66,202102,27,30500,5,2003,동일로144길 74,NaN,127.082165,37.606521,0.510661,07호선,중화
28596,서울특별시 중랑구 중화동,274-75,274.0,75.0,한영(102),72.22,202105,4,49500,4,2003,동일로144길 74,NaN,127.082165,37.606521,0.510661,07호선,중화


In [25]:
mdf.to_excel('아파트(매매)_실거래가_(역과의 거리 추가).xlsx')